In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Platform (Unified) SDK: Custom image classification model using a training pipeline with a managed dataset input

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/notebooks/deepdive/custom/ucaip_customjob_image_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/notebooks/deepdive/custom/ucaip_customjob_image_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# Overview


This tutorial demonstrates how to use the AI Platform (Unified) Python SDK to train a custom image classification model with a [training pipeline](https://cloud.google.com/ai-platform-unified/docs/training/create-training-pipeline).

### Dataset

The dataset used for this tutorial is the [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of flower an image is from a class of five flowers: daisy, dandelion, rose, sunflower, tulip.

### Objective

In this notebook, you will learn how to create a custom image classification model, using a training pipeline that that uses a managed dataset as the input and a custom container for the training step.

The steps performed include: 

- Creating a AI Platform (Unified) datasets.
- Creating a AI Platform (Unified) custom job that uses a custom training container.
- Creating a training pipeline.
- Starting the training pipeline job.
- Monitoring the training pipeline job.
- Retrieve and load the trained model artifacts.
- View the model evaluation.
- Deploy the model to a serving endpoint.
- Make a prediction(s).
- Undeploy the model.

### Costs 

This tutorial uses billable components of Google Cloud Platform (GCP):

* Cloud AI Platform
* Cloud Storage

Learn about [Cloud AI Platform
pricing](https://cloud.google.com/ml-engine/docs/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the latest (preview) version of AI Platform (Unified) SDK.

In [ ]:
%pip install -U google-cloud-aiplatform

Install cloudstorage as well.

In [ ]:
%pip install -U google-cloud-storage

Install openCV, which will be used for creating test images.

In [ ]:
%pip install -U opencv-python

### Restart the Kernel

Once you've installed the AI Platform (Unified) SDK, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select Runtime --> Change runtime type**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs, Compute Engine APIs and Container Registry API.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,containerregistry.googleapis.com)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AI Platform Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Project ID

**If you don't know your project ID**, you might be able to get your project ID using `gcloud` command by executing the second cell below.

In [1]:
PROJECT_ID = "jk-mlops-dev" #@param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
AI Platform services are
available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You can
not use a Multi-Regional Storage bucket for training with AI Platform.

In [4]:
REGION = 'us-central1' #@param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

In [6]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a custom training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. AI Platform runs
the code from this package. In this tutorial, AI Platform also saves the
trained model that results from your job in the same bucket. You can then
create an AI Platform endpoint based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

In [7]:
BUCKET_NAME = "jk-ucaip-demos" #@param {type:"string"}

In [8]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "ucaip-custom-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [9]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Creating gs://jk-ucaip-demos/...
ServiceException: 409 Bucket jk-ucaip-demos already exists.


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [10]:
! gsutil ls -al gs://$BUCKET_NAME

      1895  2020-12-04T16:09:14Z  gs://jk-ucaip-demos/trainer_cifar.tar.gz#1607098154322441  metageneration=1
                                 gs://jk-ucaip-demos/custom_job_20201204142829/
                                 gs://jk-ucaip-demos/training_data/
TOTAL: 1 objects, 1895 bytes (1.85 KiB)


### Import libraries and define constants

In [ ]:
%load_ext autoreload

#### Import AI Platform (Unified) SDK

Import the AI Platform (Unified) SDK into our python environment.

In [27]:
import json
import os
import sys
import time

import tensorflow as tf
import tensorflow_io as tfio

from google.cloud.aiplatform import gapic as aip

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

#### AI Platform (Unified) constants

Let's now setup some constants for AI Platform (Unified):

- `API_ENDPOINT`: The AI Platform (Unified) API service endpoint for dataset, model, job, pipeline and endpoint services.
- `API_PREDICT_ENDPOINT`: The AI Platform (Unified) API service endpoint for prediction.
- `PARENT`: The AI Platform (Unified) location root path for dataset, model and endpoint resources.

In [12]:
# API Endpoint
API_ENDPOINT = "us-central1-aiplatform.googleapis.com"
API_PREDICT_ENDPOINT = "us-central1-prediction-aiplatform.googleapis.com"

# AI Platform (Unified) location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

# Default timeout for API calls
TIMEOUT = 60

# Image dataset type
IMAGE_SCHEMA = 'google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml'
ANNOTATION_SCHEMA = 'gs://google-cloud-aiplatform/schema/dataset/annotation/image_classification_1.0.0.yaml'

#Image labeling type
IMPORT_SCHEMA_IMAGE_CLASSIFICATION = 'gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml'


# Training pipeline custom task defintion
CUSTOM_TASK_GCS_PATH = "gs://google-cloud-aiplatform/schema/trainingjob/definition/custom_task_1.0.0.yaml"

#### Custom Job constants

##### Hardware Accelerators

Let's now set the hardware accelerators (e.g., GPU), if any, for training and prediction.

Set the variable `TRAIN_GPU/TRAIN_NGPU` and `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the compute instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each compute instance, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

For GPU, available accelerators include:
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

   
Otherwise specify (`None, None`) for the container image for a CPU.
   
*Note*, TF releases before 2.3 for GPU support will fail to load the custom model in this tutorial. It is a known issue and fixed in TF 2.3 -- which is caused by static graph ops that are generated in the serving function. If you encounter this issue on your own custom models, the workaround is to create your own docker container image for TF 2.3 with GPU support.

In [92]:
TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

##### Container image names


In [103]:
TRAIN_IMAGE = f'gcr.io/{PROJECT_ID}/image_classifier:v1'
DEPLOY_VERSION = 'tf2-cpu.2-2'
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)
print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

Training: gcr.io/jk-mlops-dev/image_classifier:v1 AcceleratorType.NVIDIA_TESLA_K80 1
Deployment: gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest None None


#### Machine Type

Next, you will set the machine type (compute instance) you will use for training and prediction.

- Set the variables `TRAIN_COMPUTE` and `DEPLOY_COMPUTE` to the compute instance you will use for training and prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memoryn per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]
 
*Note, the following is not supported for training*
 
 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs
 
*Note, you may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*

In [15]:
MACHINE_TYPE = 'n1-standard'
VCPU='4'
TRAIN_COMPUTE = MACHINE_TYPE + '-' + VCPU
print('Train Compute Instance', TRAIN_COMPUTE)

MACHINE_TYPE = 'n1-standard'
VCPU='4'
DEPLOY_COMPUTE = MACHINE_TYPE + '-' + VCPU
print('Deploy Compute Instance', DEPLOY_COMPUTE)

Train Compute Instance n1-standard-4
Deploy Compute Instance n1-standard-4


# Tutorial

Now you are ready to start creating your own custom model and training using a pipeline for CIFAR10.



## Creating a dataset


Create a dataset client

In [16]:
client_options = {"api_endpoint": API_ENDPOINT}

dataset_client = aip.DatasetServiceClient(
        client_options=client_options
    )

### Create a dataset resource

In [19]:
display_name = 'flowers-full'

In [ ]:
dataset = aip.Dataset(
    display_name=display_name,
    metadata_schema_uri=f'gs://{DATA_SCHEMA}',
    labels=None
)

operation = dataset_client.create_dataset(parent=PARENT, dataset=dataset)
print("Long running operation:", operation.operation.name)
response = operation.result(timeout=TIMEOUT)
response

### List all datasets with a given display name

In [20]:
filter = f'display_name="{display_name}"'

request = {
    'filter': filter,
    'parent': PARENT
}

for dataset in dataset_client.list_datasets(request):
    print(f'Display name: {dataset.display_name}, Full name: {dataset.name}')
  

Display name: flowers-full, Full name: projects/895222332033/locations/us-central1/datasets/1172730306092859392


### Import data

In [ ]:
config = [{
    'gcs_source': {'uris': [IMPORT_FILE]},
    'import_schema_uri': IMPORT_SCHEMA_IMAGE_CLASSIFICATION
}]

operation = dataset_client.import_data(name=dataset.name, import_configs=config)
print("Long running operation:", operation.operation.name)
result = operation.result()

### List data items

In [ ]:
count = 0
response = dataset_client.list_data_items(parent=dataset.name)
    
for data_item in response:
    count += 1

print('Number of images: {}'.format(count))
print('An example of item specification:')
print(data_item)


## Training a model 


Now that you have seen the basic steps for custom training, you will do a new custom job to train a model. There are two ways you can train a custom model using a container image:

- **Use a Google Cloud prebuilt container**. If you use a prebuilt container, you will additionally specify a Python package to install into the container image. This Python package contains your code for training a custom model.

- **Use your own custom container image**. If you use your own container, the container needs to contain your code for training a custom model.


In this tutorial, you will train a CIFAR10 model using a Google Cloud prebuilt container.You will need to update the worker pool specification by adding a description for `python_package_spec`. This section will tell the custom job the Python training package to install and which Python module to invoke, along with command line arguments for the Python module.

Let's dive deeper now into the python package specification:

-`executor_image_spec`: This is the docker image which is configured for your custom training job. You will continue to use the same one we used earlier for demonstration.

-`package_uris`: This is a list of the locations (URIs) of your python training packages to install on the provisioned instance. The locations need to be in a Cloud Storage bucket. These can be either individual python files or a zip (archive) of an entire package. In the later case, the job service will unzip (unarchive) the contents into the docker image.

-`python_module`: The python module (script) to invoke for running the custom training job. In this example, you will be invoking `trainer.task.py` -- note that it was not neccessary to append the `.py` suffix.

-`args`: The command line arguments to pass to the corresponding pythom module. In this example, you will be:
  - `"--model-dir=" + MODEL_DIR` : The Cloud Storage location where to store the model artifacts.
  - `"--epochs=" + EPOCHS`: The number of epochs for training.
  - `"--steps=" + STEPS`: The number of steps (batches) per epoch.
  - `"--distribute=" + TRAIN_STRATEGY"` : The training distribution strategy to use for single or distributed training.
     - `"single"`: single device.
     - `"mirror"`: all GPU devices on a single compute instance.
     - `"multi"`: all GPU devices on all compute instances.

### Preparing a training container

#### Create a training package

Make the training subfolder.

In [21]:
! rm -fr trainer; mkdir trainer
! touch trainer/__init__.py

Write the training module.

In [73]:
%%writefile trainer/task.py
# Copyright 2020 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and

import argparse
import os
import time

#import hypertune
import numpy as np
import pandas as pd
import tensorflow as tf


IMG_HEIGHT = 224
IMG_WIDTH = 224
AUTOTUNE = tf.data.experimental.AUTOTUNE

    
def build_model(num_layers, dropout_ratio, num_classes):
    """
    Creates a custom image classificatin model using ResNet50 
    as a base model.
    """
    
    # Create the base model
    IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet',
                                                   pooling='avg')
    base_model.trainable = False
    
    # Add preprocessing and classification head
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.Dense(num_layers, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout_ratio)(x)
    outputs = tf.keras.layers.Dense(num_classes)(x)
    
    # Assemble the model
    model = tf.keras.Model(inputs, outputs)
    
    # Compile the model
    base_learning_rate = 0.0001
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model


def image_dataset_from_aip_jsonl(pattern, class_names=None, img_height=224, img_width=224):
        """
        Generates a `tf.data.Dataset` from a set of JSONL files
        in the AI Platform image dataset index format. 
        
        Arguments:
            pattern: A wildcard pattern for a list of JSONL files.
                E.g. gs://bucket/folder/training-*.
            class_names: the list of class names that are expected
                in the passed index. 
            img_height: The height of a generated image
            img_width: The width of a generated image
            
        """
        
        def _get_label(class_name):
            """
            Converts a string class name to an integer label.
            """
            one_hot = class_name == class_names
            return tf.argmax(one_hot)
        
        def _decode_img(file_path):
            """
            Loads an image and converts it to a resized 3D tensor.
            """
            
            img = tf.io.read_file(file_path)
            img = tf.io.decode_image(img, 
                                     expand_animations=False)
            img = tf.image.resize(img, [img_height, img_width])
            
            return img
            
        def _process_example(file_path, class_name):
            """
            Creates a converted image and a class label from
            an image path and class name.
            """
            
            label = _get_label(class_name)
            img = _decode_img(file_path)
            
            return img, label
        
        # Read the JSONL index to a pandas DataFrame
        df = pd.concat(
            [pd.read_json(path, lines=True) for path in tf.io.gfile.glob(pattern)],
            ignore_index=True
        )
        
        # Parse classifcationAnnotations field
        df = pd.concat(
            [df, pd.json_normalize(df['classificationAnnotations'].apply(pd.Series)[0])], axis=1)
        
        paths = df['imageGcsUri'].values
        labels = df['displayName'].values
        inferred_class_names = np.unique(labels)
        
        if class_names is not None:
            class_names = np.array(class_names).astype(str)
            if set(inferred_class_names) != set(class_names):
                raise ValueError(
                    'The `class_names` passed does not match the '
                    'names in the image index '
                    'Expected: %s, received %s' %
                    (inferred_class_names, class_names))
            
        class_names = tf.constant(inferred_class_names)
        
        dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
        dataset = dataset.shuffle(len(labels), reshuffle_each_iteration=False)
        dataset = dataset.map(_process_example, num_parallel_calls=AUTOTUNE)
        dataset.class_names = class_names
        
        return dataset


def get_datasets(batch_size, img_height, img_width):
    """
    Creates training and validation splits as tf.data datasets
    from an AI Platform Dataset passed by the training pipeline.
    """
    
    def _configure_for_performance(ds):
        """
        Optimizes the performance of a dataset.
        """
        ds = ds.cache()
        ds = ds.prefetch(buffer_size=AUTOTUNE)
        return ds
    
    if  os.environ['AIP_DATA_FORMAT'] != 'jsonl':
        raise RuntimeError('Wrong dataset format: {}. Expecting - jsonl'.format(
            os.environ['AIP_DATA_FORMAT']))
        
    train_ds = image_dataset_from_aip_jsonl(
        pattern=os.environ['AIP_TRAINING_DATA_URI'],
        img_height=img_height,
        img_width=img_width)
    
    class_names = train_ds.class_names.numpy()
        
    valid_ds = image_dataset_from_aip_jsonl(
        pattern=os.environ['AIP_VALIDATION_DATA_URI'],
        class_names=class_names,
        img_height=img_height,
        img_width=img_width)
    
    train_ds = _configure_for_performance(train_ds.batch(batch_size))
    valid_ds = _configure_for_performance(valid_ds.batch(batch_size))
    
    return train_ds, valid_ds, class_names
        
    
def get_args():
    """
    Returns parsed command line arguments
    """
    
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--num-epochs',
        type=int,
        default=20,
        help='number of times to go through the data, default=20')
    parser.add_argument(
        '--batch-size',
        default=32,
        type=int,
        help='number of records to read during each training step, default=32')
    parser.add_argument(
        '--num-layers',
        default=64,
        type=int,
        help='number of hidden layers in the classification head , default=64')
    parser.add_argument(
        '--dropout-ratio',
        default=0.5,
        type=float,
        help='dropout ration in the classification head , default=128')
    parser.add_argument(
        '--model-dir',
        type=str,
        default='/tmp/saved_model',
        help='model dir , default=/tmp/saved_model')

    args, _ = parser.parse_known_args()
    return args


if __name__ == "__main__":
    
    if 'AIP_DATA_FORMAT' not in os.environ:
        raise RuntimeError('No dataset information available.')
   
    args = get_args()
    
    # Create the datasets and the model
    train_ds, valid_ds, class_names = get_datasets(args.batch_size, IMG_HEIGHT, IMG_WIDTH)
    model = build_model(args.num_layers, args.dropout_ratio, len(class_names))
    print(model.summary())
    
    # Start training
    history = model.fit(x=train_ds, 
                        validation_data=valid_ds, 
                        epochs=args.num_epochs)
    
    # Save the model
    if 'AIP_MODEL_DIR' in os.environ:
        model_dir = os.environ['AIP_MODEL_DIR']
    else:
        model_dir = args.model_dir
    print('Saving the model to: {}'.format(model_dir))
    model.save(model_dir)



Overwriting trainer/task.py


#### Create a Dockefile

In [74]:
%%writefile Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-3

RUN pip install -U 'h5py<3.0.0'

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Overwriting Dockerfile


#### Build the container image

In [75]:
! docker build -t {TRAIN_IMAGE} .

Sending build context to Docker daemon  1.305MB
Step 1/5 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-3
 ---> 63032190efb7
Step 2/5 : RUN pip install -U 'h5py<3.0.0'
 ---> Using cache
 ---> 2f87e48af20c
Step 3/5 : WORKDIR /
 ---> Using cache
 ---> 9a72a4702fe6
Step 4/5 : COPY trainer /trainer
 ---> 6092706c8e4d
Step 5/5 : ENTRYPOINT ["python", "-m", "trainer.task"]
 ---> Running in d783baa07afb
Removing intermediate container d783baa07afb
 ---> 3369e1a2f036
Successfully built 3369e1a2f036
Successfully tagged gcr.io/jk-mlops-dev/image_classifier:v1


#### Push the image to Container Registry

In [76]:
! docker push {TRAIN_IMAGE}

The push refers to repository [gcr.io/jk-mlops-dev/image_classifier]

61e568df: Preparing 
adb78a68: Preparing 
cc594c09: Preparing 
4fcc8c16: Preparing 
97440069: Preparing 
609e8d31: Preparing 
53f3ad5d: Preparing 
8a31e4c1: Preparing 
095938e9: Preparing 
09c34bb9: Preparing 
cc1dc001: Preparing 
9efefae2: Preparing 
43b4c186: Preparing 
6960264e: Preparing 
1da8642f: Preparing 
5025454f: Preparing 
088bc1ab: Preparing 
a9b591f5: Preparing 
2e80fdc9: Preparing 
00c31be3: Preparing 
18b890fc: Preparing 
a7c9e3d1: Preparing 
4dce1444: Preparing 
30bcc944: Preparing 
e116c0c0: Preparing 
4df0ad6c: Preparing 
df553184: Preparing 
1e568df: Pushed lready exists 9kBv1: digest: sha256:6742e5ec0f6262b9fe1d98160e4af09d032447d0fe7259cbff487fd0a22007d7 size: 6197


### Creating a training pipeline

#### Set the pipeline's parameters

In [109]:
# REMOVE WHEN DONE TESTING
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
# TESTING

job_name = "custom_container_" + TIMESTAMP
base_output_dir = 'gs://{}/{}/'.format(BUCKET_NAME, job_name)
dataset_output_uri_prefix = base_output_dir
pipeline_display_name = 'custom-pipeline-' + TIMESTAMP
#dataset_id = dataset.name.split('/')[-1]
dataset_id = '1172730306092859392'
#dataset_id = '1047755416433328128'

epochs = 5

#### Define the container specification

In [110]:
container_spec = {
    "image_uri": TRAIN_IMAGE, 
    "args": [
        "--num-epochs=" + str(epochs),
    ],
}

container_spec

{'image_uri': 'gcr.io/jk-mlops-dev/image_classifier:v1',
 'args': ['--num-epochs=5']}

#### Define the worker pool specification

In [111]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU
    }
else:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_count": 0
    }


worker_pool_specs = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "container_spec": container_spec
    }
]

worker_pool_specs

[{'replica_count': 1,
  'machine_spec': {'machine_type': 'n1-standard-4',
   'accelerator_type': <AcceleratorType.NVIDIA_TESLA_K80: 1>,
   'accelerator_count': 1},
  'container_spec': {'image_uri': 'gcr.io/jk-mlops-dev/image_classifier:v1',
   'args': ['--num-epochs=5']}}]

#### Assemble the job specification

In [112]:
job_spec = {
    'worker_pool_specs': worker_pool_specs,
    'base_output_directory': {
        'output_uri_prefix': base_output_dir
    }
}

job_spec

{'worker_pool_specs': [{'replica_count': 1,
   'machine_spec': {'machine_type': 'n1-standard-4',
    'accelerator_type': <AcceleratorType.NVIDIA_TESLA_K80: 1>,
    'accelerator_count': 1},
   'container_spec': {'image_uri': 'gcr.io/jk-mlops-dev/image_classifier:v1',
    'args': ['--num-epochs=5']}}],
 'base_output_directory': {'output_uri_prefix': 'gs://jk-ucaip-demos/custom_container_20201207210024/'}}

#### Define the training pipeline specification

In [113]:
model_display_name = pipeline_display_name + '-model'
training_task_inputs = json_format.ParseDict(job_spec,
                                             Value())
training_pipeline_spec = {
    'display_name': pipeline_display_name,
    'input_data_config': {
        'dataset_id': dataset_id,
        'annotation_schema_uri': ANNOTATION_SCHEMA,
        'gcs_destination': {
            'output_uri_prefix': dataset_output_uri_prefix
        },
        'fraction_split': {
            'training_fraction': 0.5,
            'validation_fraction': 0.2
        },
    },
    'training_task_definition': CUSTOM_TASK_GCS_PATH,
    'training_task_inputs': training_task_inputs,
    'model_to_upload': {
        'display_name': model_display_name,
        'container_spec': {
            'image_uri': DEPLOY_IMAGE
        }
    }
}

#### Start the training pipeline 

Create a training pipeline client.

In [115]:
client_options = {"api_endpoint": API_ENDPOINT}

pipeline_client = aip.PipelineServiceClient(
    client_options=client_options)

Submit pipeline job.

In [116]:
pipeline = pipeline_client.create_training_pipeline(
    parent=PARENT, training_pipeline=training_pipeline_spec)

Get pipeline info.

In [126]:
response = pipeline_client.get_training_pipeline(name=pipeline.name)
print("pipeline")
print(" name:", response.name)
print(" display_name:", response.display_name)
print(" state:", response.state)
#print(" training_task_definition:", response.training_task_definition)
#print(" training_task_inputs:", dict(response.training_task_inputs))
print(" create_time:", response.create_time)
print(" start_time:", response.start_time)
print(" end_time:", response.end_time)
print(" update_time:", response.update_time)
print(" labels:", dict(response.labels))

pipeline
 name: projects/895222332033/locations/us-central1/trainingPipelines/1329142432213762048
 display_name: object-detection-demo_2020123172535
 state: PipelineState.PIPELINE_STATE_SUCCEEDED
 create_time: 2020-12-03 17:26:39.045464+00:00
 start_time: 2020-12-03 17:29:33.827510+00:00
 end_time: 2020-12-03 20:20:07.063286+00:00
 update_time: 2020-12-03 20:20:07.063286+00:00
 labels: {}


## Deploying


In [169]:
if pipeline.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
    print("Training job has not completed")
else:
    print("Pipeline completed")

Pipeline completed


### List all models

Now that your custom model is uploaded as a AI Platform (Unified) managed model, let's get a list of all your AI Platform (Unified) managed models. Use this helper function `list_models`. This helper function uses the AI Platform (Unified) model client service, and calls the method `list_models`, with the parameter:

- `parent`: The AI Platform (Unified) location root path for your dataset, model and endpoint resources.

The response object from the call is a list, where each element is a AI Platform (Unified) managed model. For each model, you will display a few fields:

- `name`: The AI Platform (Unified) unique identifier for the managed model.
- `display_name`: The human readable name assigned to the model.
- `create_time`': Timestamp when the model resource was created.
- `update_time`: Timestamp when the model resource was last updated.
- `container`: The container image used for training the model.
- `artifact_uri`': The Cloud Storage location of the model artifact.

In [132]:
client_options = {"api_endpoint": API_ENDPOINT}

model_client = aip.ModelServiceClient(
    client_options=client_options)

In [170]:
response = model_client.list_models(parent=PARENT)
for model in response:
    print("name", model.name)
    print("display_name", model.display_name)
    print("create_time", model.create_time)
    print("update_time", model.update_time)
    print("container", model.container_spec.image_uri)
    print("artifact_uri", model.artifact_uri)
    print('\n')

name projects/895222332033/locations/us-central1/models/2679747331401711616
display_name cifar10-20201208003332
create_time 2020-12-08 01:01:13.309475+00:00
update_time 2020-12-08 01:01:32.461794+00:00
container gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest
artifact_uri gs://jk-ucaip-demos/custom_job_20201208003332


name projects/895222332033/locations/us-central1/models/1109468805077139456
display_name cifar10-training-pipeline-20201208003332-model
create_time 2020-12-08 00:34:55.563786+00:00
update_time 2020-12-08 00:47:47.397380+00:00
container gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest
artifact_uri gs://jk-ucaip-demos/custom_job_20201208003332


name projects/895222332033/locations/us-central1/models/9115180092681617408
display_name custom-pipeline-20201207202948-model
create_time 2020-12-07 20:56:01.300299+00:00
update_time 2020-12-07 21:12:23.132470+00:00
container gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest
artifact_uri gs://jk-ucaip-demos/custom_co

#### List all models with a given display name

In [173]:
request = {
    'parent': PARENT,
    'filter': 'display_name="{}"'.format(model_display_name)
}

response = model_client.list_models(request)

for model in response:
    print(model.name)

### Get model information

Now let's get the model information for just your model. Use this helper function `get_model`, with the parameter:

- `name`: The AI Platform (Unified) unique identifier for the managed model.

This helper function uses the AI Platform (Unified) model client service, and calls the method `get_model`, with the parameter:

- `name`: The AI Platform (Unified) unique identifier for the managed model.

In [ ]:
def get_model(name):
    response = clients['model'].get_model(name=name)
    print(response)


get_model(model_to_deploy_name)

### Create an endpoint

Use this helper function `create_endpoint` to create an endpoint to deploy the model to for serving predictions, with the parameter:

- `display_name`: A human readable name for the endpoint.

The helper function uses the endpoint client service and calls the method `create_endpoint`, which takes the parameter:

- `display_name`: A human readable name for the endpoint.

Creating an endpoint returns a long running operation, since it may take a few moments to provision the endpoint for serving. You call `response.result()`, which is a synchronous call and will return when the endpoint is ready. The helper function will return the AI Platform (Unified) fully qualified identifier for the endpoint -- `response.name`.


In [167]:
client_options = {"api_endpoint": API_ENDPOINT}

endpoint_client = aip.EndpointServiceClient(
    client_options=client_options)

In [168]:
for endpoint in endpoint_client.list_endpoints(parent=PARENT):
    print(endpoint)
    print('*')

name: "projects/895222332033/locations/us-central1/endpoints/4464509789979803648"
display_name: "cifar10_endpoint-20201208003332"
etag: "AMEw9yPY6vNMTmAOPY4tfeZDC1VrMbssuDytuRvby1qye75a5j8O_oPF4SidF-WU_uA="
create_time {
  seconds: 1607389368
  nanos: 582338000
}
update_time {
  seconds: 1607389385
  nanos: 244946000
}

*


In [161]:
API_ENDPOINT

'us-central1-aiplatform.googleapis.com'

In [162]:
PARENT

'projects/jk-mlops-dev/locations/us-central1'

In [166]:
endpoint_name = 'projects/895222332033/locations/us-central1/endpoints/6276082740089585664'

endpoint_client.get_endpoint(name=endpoint_name)

NotFound: 404 Endpoint projects/895222332033/locations/us-central1/endpoints/6276082740089585664 is not found.

In [ ]:
ENDPOINT_NAME = "cifar10_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients['endpoint'].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result.name


endpoint_name = create_endpoint(ENDPOINT_NAME)

### Deploy model to the endpoint

Use this helper function `deploy_model` to deploy the model to the endpoint you created for serving predictions, with the parameters:

- `model`: The AI Platform (Unified) fully qualified model identifier of the model to upload (deploy) from the training pipeline.
- `deploy_mopdel_display_name`: A human readable name for the deployed model.
- `endpoint`: The AI Platform (Unified) fully qualified endpoint identifier to deploy the model to.

The helper function uses the endpoint client service and calls the method `deploy_model`, which takes the parameters:

- `endpoint`: The AI Platform (Unified) fully qualified endpoint identifier to deploy the model to.
- `deployed_model`: The requirements for deploying the model.
- `traffic_split`: Percent of traffic at endpoint that goes to this model, which is specified as a dictioney of one or more key/value pairs.
   - If only one model, then specify as { "0": 100 }, where "0" refers to this model being uploaded and 100 means 100% of the traffic.
   - If there are existing models on the endpoint, for which the traffic will be split, then specify as, where `model_id` is the model id of an existing model to the deployed endpoint. The percents must add up to 100.
   
           { "0": percent, model_id: percent, ... }

Let's now dive deeper into the `deployed_model` parameter. This parameter is specified as a Python dictionary with the minimum required fields:

- `model`: The AI Platform (Unified) fully qualified model identifier of the (upload) model to deploy.
- `display_name`: A human readable name for the deployed model.
- `dedicated_resources`: This refers to how many redundant compute instances (replicas) and type of compute instance (machine_spec). For this example, we set it to one (no replication). If using a GPU, the corresponding container image must support a GPU.

Let's now dive deeper into the `traffic_split` parameter. This parameter is specified as a python dictionary. This might at first be a tad bit confusing. Let me explain, you can deploy more than one instance of your model to an endpoint, and then set how much (percent) goes to each instance. 

Why would you do that? Perhaps you already have a previous version deployed in production -- let's call that v1. You got better model evaluation on v2, but you don't know for certain that it is really better until you deploy to production. So in the case of traffic split, you might want to deploy v2 to the same endpoint as v1, but it only get's say 10% of the traffic. That way, you can monitor how well it does without disrupting the majority of users -- until you make a final decision.

In [ ]:
DEPLOYED_NAME = "cifar10_deployed-" + TIMESTAMP


def deploy_model(model, deployed_model_display_name, endpoint, traffic_split={"0": 100}):
    # Accelerators can be used only if the model specifies a GPU image.
    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        # `dedicated_resources` must be used for non-AutoML models
        "dedicated_resources": {
            "min_replica_count": 1,
            "machine_spec": machine_spec
        },
    }

    response = clients['endpoint'].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split)

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_name, DEPLOYED_NAME, endpoint_name)

### List all endpoints

Let's now get a list of all your endpoints. Use this helper function `list_endpoints`. 

The helper function uses the endpoint client service and calls the method `list_endpoints`. The returned response object is a list, with an element for each endpoint. The helper function lists a few example fields for each endpoint:

- `name`: The AI Platform (Unified) identifier for the managed endpoint.
- `display_name`: The human readable name you assigned to the endpoint.
- `create_time`: When the endpoint was created.
- `deployed_models`: The models and associated information that are deployed to this endpoint.

In [ ]:
def list_endpoints():
    response = clients['endpoint'].list_endpoints(parent=PARENT)
    for endpoint in response:
        print("name:", endpoint.name)
        print("display name:", endpoint.display_name)
        print("create_time:", endpoint.create_time)
        print("deployed_models", endpoint.deployed_models)
        print("\n")
        
list_endpoints()

### Get information on this endpoint

Now let's get the endpoint information for just your endpoint. Use this helper function `get_endpoint`, with the parameter:

- `name`: The AI Platform (Unified) unique identifier for the managed endpoint.

This helper function uses the AI Platform (Unified) endpoint client service, and calls the method `get_endpoint`, with the parameter:

- `name`: The AI Platform (Unified) unique identifier for the managed endpoint.

In [ ]:
def get_endpoint(name):
    response = clients['endpoint'].get_endpoint(name=name)
    print(response)
    
get_endpoint(endpoint_name)

## Make a prediction request

Let's now do a prediction to your deployed model. You will use an arbitrary image out of the test (holdout) portion of the dataset as a test image. 

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

### Prepare the request content
You are going to send the CIFAR10 image as compressed JPG image, instead of the raw uncompressed bytes:

- `cv2.imwrite`: Use openCV to write the uncompressed image to disk as a compressed JPEG image.
- `tf.io.read_file`: Read the compressed JPG images back into memory as raw bytes.
- `base64.b64encode`: Encode the raw bytes into a base 64 encoded string.

In [ ]:
import base64
import cv2
cv2.imwrite('tmp.jpg', (test_image * 255).astype(np.uint8))

bytes = tf.io.read_file('tmp.jpg')
b64str = base64.b64encode(bytes.numpy()).decode('utf-8')

### Send the prediction request

Ok, now you have a test image. Use this helper function `predict_image`, which takes the parameters:

- `image`: The test image data as a numpy array.
- `endpoint`: The AI Platform (Unified) fully qualified identifier for the endpoint where the model was deployed.
- `parameters_dict`: Additional parameters for serving -- in our case we will pass None.

This function uses the prediction client service and calls the `predict` method with the parameters:

- `endpoint`: The AI Platform (Unified) fully qualified identifier for the endpoint where the model was deployed.
- `instances`: A list of instances (encoded images) to predict.
- `parameters`: Additional parameters for serving -- in our case we will pass None.

To pass the image data to the prediction service, in the previous step you encoded the bytes into base 64 -- which makes the content safe from modification when transmitting binary data over the network. You need to tell the serving binary where your model is deployed to, that the content has been base 64 encoded, so it will decode it on the other end in the serving binary. 

Each instance in the prediction request is a dictionary entry of the form:

                        {input_name: {'b64': content }}
                        
- `input_name`: the name of the input layer of the underlying model.
- `'b64'`: A key that indicates the content is base 64 encoded.
- `content`: The compressed JPG image bytes as a base 64 encoded string.

Since the `predict()` service can take multiple images (instances), you will send your single image as a list of one image. As a final step, you package the instances list into Google's protobuf format -- which is what we pass to the `predict()` service.

The `response` object returns a list, where each element in the list corresponds to the corresponding image in the request. You will see in the output for each prediction:

- Confidence level for the prediction (`predictions`, between 0 and 1, for each of the ten classes.

In [ ]:
def predict_image(image, endpoint, parameters_dict):
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{input_name: {'b64': image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients['prediction'].predict(endpoint=endpoint, instances=instances, parameters=parameters_dict)
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        # See gs://google-cloud-aiplatform/schema/predict/prediction/classification.yaml for the format of the predictions.
        print(" prediction:", prediction)


predict_image(b64str, endpoint_name, None)

## Undeploy the model

Let's now undeploy your model from the serving endpoint. Use this helper function `undeploy_model`, which takes the parameters:

- `deployed_model_id`: The model deployment identifier returned by the endpoint service when the model was deployed.
- `endpoint`: The AI Platform (Unified) fully qualified identifier for the endpoint where the model is deployed.

This function uses the endpoint client service and calls the method `undeploy_model`, with the parameters:

- `deployed_model_id`: The model deployment identifier returned by the endpoint service when the model was deployed.
- `endpoint`: The AI Platform (Unified) fully qualified identifier for the endpoint where the model is deployed.
- `traffic_split`: How to split traffic among the remaining deployed models on the endpoint.

Since this is the only deployed model on the endpoint, we simply can leave `traffic_split` empty by setting it to {}.

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients['endpoint'].undeploy_model(endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={})
    print(response)


undeploy_model(deployed_model_id, endpoint_name)

# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Model
- Endpoint
- Cloud Storage Bucket

In [ ]:
delete_dataset = True
delete_model = True
delete_endpoint = True
delete_bucket = True

# Delete the dataset using the AI Platform (Unified) fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients['dataset'].delete_dataset(name=dataset['name'])
except Exception as e:
    print(e)

# Delete the model using the AI Platform (Unified) fully qualified identifier for the model
try:
    if delete_model:
        clients['model'].delete_model(name=model_to_deploy_name)
except Exception as e:
    print(e)

# Delete the endpoint using the AI Platform (Unified) fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients['endpoint'].delete_endpoint(name=endpoint_name)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME